# Trends in Permutations

A trend is a number sequence with one interesting property:
if you put your finger between any two numbers in that sequence, breaking it into a prefix and a suffix,
the average of the prefix is smaller than the average of the suffix.

In a trend, things are always "looking up." The `trendlist` package, which create and explore trends. 
If you'd like more in-depth information on this package, it's available in a sibling notebook.

A fun way to explore trends is to pick a sequence, then decompose each permutation of that sequence into trends.

This notebook explores that.

Most of the tools we'll need to do this are in the submodule `trendlist.simple`,
and we'll save time by just importing them right from the start.

In [ ]:
from trendlist.simple import *

## Unique Decomposition

You can decompose nearly every list, uniquely, into maximum-length trends.
When you do, the average of those trends drops, steadily, from left-to-right.

For example, [32, 4, 1, 16, 2, 8] decomposes into [32] [4, 1, 16] [2, 8]. Each subsequence is a trend, and the means of the trends drops from left to right. $\mu([32]) == 32 > \mu([4, 1, 16] == 7 > \mu([2, 8]) == 5$ 

This makes sense. After all, if a trend mean is less than the mean of the trend to its immediate right,
then the two form a single, longer trend.

In [ ]:
ts = [[32], [4, 1, 16], [2, 8]]
for t in ts: 
    print(f"{t=} {'is a trend' if is_trend(t) else 'is not a trend'}, with mean {statistics.mean(t)}")

Though tacking any pair of these trends together in their original order leaves them as two, separate trends,

In [ ]:
is_trend(ts[1] + ts[2])

reversing the order, so that the left trend's mean is less than the right's, turns that concatenation into a single trend.

In [ ]:
is_trend(ts[2]+ts[1])

However, not all sequences decompose uniquely. How would you decompose [1, 1, 1, 1, 1]? 

* As a single trend, [1, 1, 1, 1, 1]? 
* As two, [1,1] [1, 1, 1]? 
* As five, [1], [1], [1], [1], [1]?

For unique decomposition, you only need a couple of conditions.

In [ ]:
is_trend([1,1,1,1])

1. **in-between-ness**
If you break a sequence in two, the two parts must have averages that flank the average of the whole;
that is, if A and B are two sub-sequences, and A+B is their concatenation, then average(A) > average(B) 
implies average(A) > average(A+B) > average(B). 

As long as you can pick some sort of "average" which guarantees that the average of any pair of collections lies strictly between the average of each, it'll work.

The arithmetic mean, $\mu$, has in-between-ness, but so do the geometric mean, the harmonic mean, and many other measures of central tendency.

In [ ]:
from statistics import mean as arithmetic_mean
from statistics import geometric_mean, harmonic_mean
A = [1, 1, 2, 2, 2, 3]
B = [1, 1, 4, 4, 4, 5]
for mean in arithmetic_mean, geometric_mean, harmonic_mean:
    print(mean)
    print(f"{mean(A)=:0.2f}")
    print(f"{mean(A+B)=:0.2f}")
    print(f"{mean(B)=:0.2f}")

Another common measure of central tendency, the mode (the most frequent value) doesn't work here!

In [ ]:
from statistics import mode
print(f"{mode(A)=:0.2f}")
print(f"{mode(A+B)=:0.2f}")
print(f"{mode(B)=:0.2f}")

2. **uniqueness** No two different trends can have the same average: if they do, the order of the two trends in the decomposition is ambiguous.

Medians regularly runs afoul of this condition.

In [ ]:
from statistics import median
A = [1, 2, 3, 10]
B = [0, 2.2, 2.8, 100]
print(f"{median(A)=:0.2f}")
print(f"{median(A+B)=:0.2f}")
print(f"{median(B)=:0.2f}")

Thus, we could use any of the means, but not the median or the mode for our "average." We'll use the arithmetic mean because it's familiar, and dead-easy to compute.

Next, notice that if every subset of a set of numbers has a different average, 
then any permutation of those numbers has a unique decomposition,
because no matter how you order those numbers, 
no two, neighboring trends in their decomposition could ever wind up with the same average.

When you can assure that, adjacent trends in *any* permutation of those numbers can never have the same average. 

Let's call an iterable where you can uniquely decompose any permutation, ***trendy***.

I suspect two kinds of sequences will satisfy both conditions.

1. Sets of prime powers: sets like ${1, 5, 25, 125, ..., 5^k}$
1. Sets of random reals

$\color{green}{\text{A proof of my suspicion would be nice, but I don't have one yet.}}$

Meanwhile, `trendlist.simple.pows()` will generate the first, and `trendlist.rands()`, the second.

In everything that follows we'll use these two kinds of sequences and to the arithmetic mean, which is familiar and easy to compute.

Next, let's move on to counting the number of trends in every permutation of a trendy iterable.
Perhaps you're saying to everyone who's around you, "This is just a problem in elementary enumerative combinatorics!"

Or, perhaps not. :-)

**[Spoiler alert: I'm not.]**

Even if you're the kind of person who says things like that, perhaps it's not obvious yet how to attack it.

Let's explore a little, to see if we can get ideas by writing some code.

## Counting Trends in Permutations

The set `pows(3)` = ${1, 2, 4}$, has $3! = 6$ possible permutations.
You can generate all $6$ with `itertools.permutations()` like this:

In [ ]:
import itertools

def perms(s):
    # return a list of all the permutations of the elements of s
    # s can be any iterable
    _perms = itertools.permutations(s) # _perms generates tuples
    _perms = [list(perm) for perm in _perms] # but lists are easier to work with
    return _perms

In [ ]:
perms(pows(3))

`pows()` returns a list, but `perms` will accept a list, a tuple, a set, or even a generator.

Next, let's decompose each permutation into trends.
The `trendlist` package offers efficient algorithms and useful data structures for these tasks,
but in what follows, we'll mostly use the brute-force routines defined in the submodule `trendlist.simple`.
They're easy to read, understand, and use.

To decompose a list into trends, you can just use `trend_list()`

In [ ]:
s = [2, 4, 1]
trend_list(s)

The data structures, functions, and methods are all discussed in a separate notebook, [The `trendlist` Package]()]

We can decompose all the permutations in a simple loop.

In [ ]:
def print_all_trend_lists(s):
    for perm in perms(s):
        print(f"{perm=} -> {trend_list(perm)}")

print_all_trend_lists(pows(4))

Go ahead and try `print_all_trend_lists()` with your own, trendy enumerable.

We already mentioned, in passing, that the trend means in a trend list drop from left to right.
Before going any farther, notice a couple of other things:

1. Swapping any pair of adjacent trends gives you a new permutation that has exactly one fewer trend.
If you have a trendlist with $n$ trends, $L = [T_1, T_2, ... T_{k-1}, T_k, T_{k+1}, T_{k+2}, ... T_n]$,
with means $M(L) = [\mu_1, \mu_2, ... \mu_n]$,
and you swap $T_k$ and $T_{k+1}$, these trends will merge into a single trend, $T_x$.
But because $\mu_{k-1} > \mu_k > \mu_x > \mu_{k+1} > \mu_{k+2}$, 
fusing that pair just drops the number of trends by 1:
$L2 = [T_1, T_2, ... T_{k-1}, T_x, T_{k+2}, ... T_n]$

1. Rotating a trend from one end to the other decreases the number of trends, by *at least one*, but maybe more.
In the trendlist $L$, if, instead of swapping a pair of neighbors, you rotate $T_n$ from the right end to the beginning, $T_n$ and $T_1$ will fuse into a single trend, $T_y$ because $\mu_n$ < $\mu_1$.  

But you might not be done!

You know that $\mu_1 > \mu_2$. But $\mu_y$ is somewhere between $\mu_1$ and $\mu_n$. If it's a lot smaller than $\mu_1$ then it might be smaller than $\mu_2$, so you have to merge $T_y$ with $T_2$, and so on.

In the example above, if you start with the permutation $p = [2, 8, 4, 1]$, decompose it into its trends $L = [[2, 8], [4], [1]]$, rotate $[1]$ to the beginning, and let the dust settle, you end up with a single trend: $L' = [[1, 2, 8, 4]]$

In other words,

* If you keep rotating trends from one end to the other, one at a time, each rotation will decrease the number of trends by *some* number, $>= 1$, until you can't rotate any more, because you finally have a single trend.

This algorithm -- decompose into trends with trend_list, then rotate until you have a single trend -- shows that every sequence has a circular permutation that's a single trend.  It's not a huge amount of work to show that no *other* circular permutation is a single trend -- that no permutation of a single trend is still a trend -- but we'll leave that exercise to you.

Here's the important take-home:

**Every sequence from a trendy set has *exactly one* circular permutation that is a single trend.**

## Counting trends

How many trends does each trend_list have?

In [ ]:
def all_trend_lists(seq, verbose=False):
    # decompose *every* permutation of seq into trends
    return [trend_list(perm) for perm in perms(seq)]

def count_trends(list_of_trend_lists, verbose=False):
    # count the # of trends in each trend_list
    if verbose:
        for trend_list in list_of_trend_lists:
            print(f"{trend_list=} has {len(trend_list)} trend(s)")
    return [len(trend_list) for trend_list in list_of_trend_lists]

seq = pows(3)
count_trends(all_trend_lists(seq), verbose=True)

And how frequent is each of these counts? We can summarize them with `collections.Counter()`, which puts all the $0$ trends into one bin, all the $1$ trends in another, and so on.

In [ ]:
from collections import Counter

counts = Counter(count_trends(all_trend_lists(seq)))

Having collected and binned those data, you can report what you found:

In [ ]:
for count in range(len(seq)+2):
    print(f"number of permutations with exactly {count} trends is {counts[count]}")

Note that `collections.Counter()` cheerfully reports counts of $0$ for everything it doesn't know about,
instead of giving you an error.

In [ ]:
print(f"the number of permutations with exactly 100 trends is {counts[100]}")

Before moving on, wrap that into a function.

In [ ]:
def n_trends(s):
    # Return list telling how many permutations of s have exactly k trends.
    # Include 0 at the beginning to say that no permutation has *no* trends.
    # Besides, Python programmers like the first array index to be 0, not 1.
    counts = [0]*(len(s) + 1)
    for ntrends, count in Counter(count_trends(all_trend_lists(s))).items():
        counts[ntrends] = count
    return counts

n_trends(list(pows(7)))

## First, Not-Very-Deep Thoughts about `n_trends()`

You can already say a few things about `n_trends()`.

1. As long as the argument is trendy, the returned list should only depend on the size of the arg.
1. Every permutation has *some* number of trends, so it's in one of the buckets. This means the sum of the list `n_trends()` returns will be $n!$

Let's see those:

In [ ]:
from trendlist import rands # sequences from a different trendy set
# 1. trendy values don't matter, only their lengths
n = 7
r = list(rands(7))
p = pows(7)
n_trends(r) == n_trends(p)

In [ ]:
# 2. n_trends(s) sums to len(s)!
from math import factorial

# n_trends(s) sums to len(s)!
p = pows(8)
print(f"sum of all buckets is n!: {sum(n_trends(p)) == factorial(len(p))}")

Also,

3. Every list has exactly one circular permutation that's a single trend.
This partitions the $n!$ permutations into non-overlapping subsets, each of size $n$, that can be rotated into the same single trend.

Using our earlier example, the list [1, 2, 8, 4] has four circular permutations.
$[1, 2, 8, 4]$

$[4, 1, 2, 8]$

$[8, 4, 1, 2]$

$[2, 8, 4, 1]$


Each can broken down into a trendlist.

$[[1, 2, 8, 4]]$

$[[4], [1, 2, 8]]$

$[[8], [4], [1, 2]]$

$[[2, 8], [4], [1]]$


Exactly one of these has a lone trend.

This batching of permutations into equal-sized buckets, all of which are circular permutations of one another,
means the number of permutations that are a single trend is `n_trends()[1]` = $n!/n = (n-1)!$

This is important, and a little surprising, so let's check that it's correct:

In [ ]:
for n in range(1, 8):
    seq = pows(n)
    # n_trends(seq)[1] = (n-1)!
    print(f"for sequences of length {n}, {n-1=}: number of single trends: {n_trends(seq)[1]}")


## Slightly Beyond the Basics with `n_trends()`

Clearly `n_trends()[0]` == $0$ and we just showed that `n_trends()[1]` == $(n-1)!$.
Can we say anthing else about `n_trends()[k]`, the number of permutations with k trends?

Here's another special case: $k = n-1$.

In [ ]:
for n in range(1, 8):
    seq = pows(n)
    k = n-1
    # n_trends(seq)[-1] = 1
    print(f"when {n=}, the number of permutations with {k=} trends is {n_trends(seq)[-1]}")

Well, that's regular. Why?

Easy enough. First, any sequence that's sorted from biggest to smallest will have exactly $n$ trends: every element will be its own trend.

The first number is bigger than the average of everything that follows it, so it's a trend of length $1$.
The same argument holds, in turn, for each element in the sequence.

In [ ]:
p_7 = pows(7)
tl = trend_list(sorted(p_7, reverse=True))
print(f"{tl=}, and {len(tl)=}")

Could other permutations also have $n$ trends? No way.

1. To have $n$ trends, each element must be its own trend.
1. In a trendlist, the trends are monotonically decreasing from left to right.
1. The elements in the original sequence *must* also decrease monotonically.
1. The sequence must have been sorted, in decreasing order.

In other words, there's one and only one sequence with $n$ trends.

A similar argument shows you `n_trends()[-2]` $== {n \choose 2}$

Here it is, step-by-step:

1. `n_trends()[-2]` is the number of permutations with exactly $n-1$ trends.
1. To have exactly $n-1$ trends, exactly one trend must have 2 elements.
1. There are $n \choose 2$ ways to pick 2 elements out to put in that trend.
1. The two-element trend can only be in increasing order: [smaller, larger]. Otherwise, it's not a trend.
1. Once the elements are assigned to trends -- two to a 2-element trend in increasing order, one to each of the rest -- there is only one way to arrange the trends: with their means in decreasing order.

In other words, by picking two elements, you define a unique permutation that decomposes into $n-1$ trends, with one of those trends made up of the two elements you picked, in increasing order.

Check it out:

In [ ]:
from math import comb # binomial coefficients

for n in range(2, 8):
    seq = pows(n)
    k = n-2
    # n_trends(seq)[-2] = comb(n, k)
    print(f"when {n=}, the number of permutations with {k=} trends is {n_trends(seq)[-2]}")
    print(f"and {comb(n, k)=}")

Okay, but these are special cases: the number of permutations of a sequence with exactly $1$ trend, the number with exactly $n-1$ trendsm the number with exactly $n-2$ trends. Can we say how many permutations of a sequence of length $n$ has exactly $k$ trends, for any $n$ abd $k$?

Why yes, we can.

## `n_trends()[k]`, the number of permutations with exactly `k` trends.

## `n_trends(s)[k]` is $len(s) \brack k$

Is there a simple expression for the number of permutations of `s` with exactly `k` trends?
Yes!

Sort of.

First, you have to learn about Stirling Numbers.

## Stirling Numbers Count Cycles

A Sterling number of the first kind (unsigned), written $n \brack k$, 
is the number of ways you can decompose *n* elements into *k* distinct cycles.

(Folks have used are several different notations for these numbers. We're using Don Knuth's, because we have to use something and we've been Don Knuth fanboys since our teen years, some time back in the mid-Cretaceous.)

What's that mean?

A set of four elements, $\{a, b, c, d\}$, has these, distinct decompositions:

* (a b c d)
* (a b d c)
* (a c b d)
* (a c d b)
* (a d b c)
* (a d c b)
* (a)(b c d)
* (a)(b d c)
* (b)(a c d)
* (b)(a d c)
* (c)(a b d)
* (c)(a d b)
* (d)(a b c)
* (d)(a c b)
* (a b)(c d)
* (a c)(b d)
* (a d)(b c)
* (a)(b)(c d)
* (a)(c)(b d)
* (a)(d)(b c)
* (b)(c)(a d)
* (b)(d)(a c)
* (c)(d)(a b)
* (a)(b)(c)(d)

Two cycles are considered identical if they're circular permutations of one another, so 

$(a b c d) == (b c d a) == (c d a b) == (d a b c)$

Any other decomposition will fall into one of these buckets.
For example, $(a) (c d b)$ is not in the list above, but it's just a different way of writing the cycle $(a) (b c d)$.

Similarly, ordering of adjacent cycles doesn't matter, so $(a b) (c d)$ is the same as $(c d) (a b)$.

If you count elements in the list above, you can see that there are 11 lines with exactly two cycles. $ {4 \brack 2} == 11 $

This is easy to see, with reasoning analogous to that used to show `n_trends(s)[-2]` $== {n \choose 2}$

1. Partition the $n$ elements of a trendy iterable into $k$ subsets.
1. Consider each subset a cycle, and rotate it into the unique order that is a single trend.
1. Order the trends in order of decreasing means.

This is a 1-1 match of trendlists of permutations to decompositions of the set into distinct cycles, and vice-versa. It's what math guys call a *bijection*.
Each decomposition into $k$ cycles maps, reversably, to a decomposition of some permutation into $k$ trends. 

To find the number of ways to take $n$, random reals and write them as a single sequence of $k$ trends, just look up the value of $n \brack k$ .

Stirling numbers were first studied by James Stirling [1692-1770], a crony of Sir Isaac Newton's. The numbers were named for him, not the other way around.

They pop up in unexpected places. This is yet another. There isn't a simple, closed, algebraic expression for *$n \brack k$*, 
but there isn't for *$sin(\theta)$* or *$ln(x)$* either, and you've gotten used to using sines and natural logs. *Stirling number* is just another well-defined function to add to your mathematical vocabulary.

When you actually need a value for *$n \brack k$*, you just plug in the parameters and look it up, the way you would  𝑡𝑎𝑛(𝜃)  or a  𝑝 -value for a particular part of the normal distribution.

In [ ]:
print(f"Using n_trends(): {n_trends(pows(8))}")

In [ ]:
def stirlings(n):
    # compute n_trends() with Stirling numbers
    from sympy.functions.combinatorial.numbers import stirling # package for symbolic computation
    row = []
    for k in range(n+1):
        row.append(stirling(n=n, k=k, kind=1))
    return row
print(f"Using Stirling numbers: {stirlings(8)}")

## Why Does This Help?

Okay, so it's the Sterling numbers (of the first kind, unsigned). So what?

First, there's a big performance difference.

In [ ]:
for n in range(1, 6):
    print(f"n_trends(pows({n})): ", end="")
    %timeit n_trends(pows(n))
    print(f"stirlings({n}): ", end="")
    %timeit stirlings(n)


So you don't have to look it up $1ms == 1000\mu s == 1,000,000 ns$.

Don't even think about using `n_trends()` for larger *n*.

In [ ]:
%timeit stirlings(20)

Second, if we know a fact about Stirling numbers, we can apply it to trends.
For example, we know that the average Stirling number, $n \brack k$, is $H(n)$, 
the *n*th Harmonic number, $1 + 1/2 + 1/3 + ... + 1/n$

In 1734, Leonard Euler showed that $\lim\limits_{n \to \inf}(H(n) - ln(n)) = \gamma$,
where $\gamma = 0.5775664...$ is the Euler-Mascheroni constant.

In other words, the average number of trends in a sequence of length $N$ is about $ln(N)$.

Before, you could say that when you decompose sequences of 1,000,000 random reals, you'll get a single trend one time in a million. Now, you can also say that typically you'll expect roughly $ln(10^6) \approx 13.7$ trends.

## Counting Lengths of Longest Trends

Let's count something else. If you list all permutations of a sequence, decomposed into trends, how long is the longest trend? We'll cannibalize the earlier code.

In [ ]:
def longest_trend(trendlist):
    return max([len(trend) for trend in trendlist])
    
def count_longest_trends(trendlists):
    return [longest_trend(tl) for tl in trendlists]

count_longest_trends(all_trend_lists(pows(3)))

In [ ]:
from collections import Counter

counts = Counter(count_longest_trends(all_trend_lists([1, 2, 4])))

for count in range(len([1,2,4])+1):
    print(f"the number of permutations that have a longest trend of length {count} trends is {counts[count]}")

In [ ]:
def l_trends(s):
    # OEIS iiA126074
    # Return list telling how many permutations of s have a longest trend of a given length.
    # Include 0 at the beginning to say that no permutation has *no* trends,
    # because, Python programmers like the first array index to be 0, not 1.
    counts = [0]*(len(s) + 1)
    for ltrends, count in Counter(count_longest_trends(all_trend_lists(s))).items():
        counts[ltrends] = count
    return counts

l_trends(pows(8))

If you decompose each of the $8!$ permutation of `pows(8) = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]`, then 5040 have single trends of length 8, the whole sequence. 
Of course, you knew that because 1 out of every 8 is a single trend.
$8!/8 = 7! = 5040$

Now you know that $6720$ have a longest trend of length $6$!

Once again, all sequences must have *some* longest trend, so the sum of `l_trends(s)` must be `factorial(len(s))`.

Watch:

In [ ]:
n = 6
print(l_trends(pows(n)))
nlongests = sum(l_trends(pows(n)))
print(f"the total number of longest trends (`l_trends`) {'is' if nlongests == factorial(n) else 'is not'} {n}!")

So, as sequences grow longer, how much of their decomposition is in the single, longest trend? 

In [ ]:
def sum_max_trend_lengths(s):
    # OEIS A028418
    sum = 0
    for l_trend, count in enumerate(l_trends(s)):
        sum += l_trend*count
    return sum

def mean_max_trend_length(s):
    return sum_max_trend_lengths(s)/(factorial(len(s))*len(s))

for length in range(1, 10):
    print(f"{length=}: {mean_max_trend_length(pows(length))})")

It looks like the fraction continues to drop, but our brute-force package is too slow to tell whether its assymptotic value is $> 0$